In [1]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import pandas
import torch
import SimpleITK as sitk
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from torchvision import transforms, datasets
from torch.utils import data

In [3]:
DATA_PATH = 'C:/Users/Kyle/'

In [4]:
def read_data():
    data_path = DATA_PATH + 'Desktop/Python/MURA-v1.1/'
    sub_path = 'train/study1_negative/'
    train_xray = pandas.read_csv(data_path + 'train_labeled_studies.csv', header=None)
    train_xray[0] = train_xray[0].astype(str)
    humerus_xray = pandas.concat([train_xray], axis=0, ignore_index=True)

    # Split image path
    imageSplit = train_xray[0].str.split('/', n=5, expand=True)

    train_xray['ImageSet'] = imageSplit[1]
    train_xray['ImageType'] = imageSplit[2]
    train_xray['Patient ID'] = imageSplit[3]
    train_xray['DiagnosisType'] = imageSplit[4]
    train_xray['Diagnosis'] = imageSplit[4].str.split('_', n=1, expand=True)[1]

    diagnosis_label = {'negative': 0, 'positive': 1}

    patient_dict = {}
    for i in range(train_xray.shape[0]):
        patient_id = train_xray['Patient ID'][i]
        target_diagnosis = train_xray['Diagnosis'][i]
        target_diagnosis = diagnosis_label[target_diagnosis]
        patient_dict[patient_id] = target_diagnosis

    for file_name in tqdm(os.listdir(data_path + sub_path +
                                     train_xray['Patient ID'][0] + '/' + train_xray['DiagnosisType'][0])):
        image_filename = file_name.split('.')[0]
        target_diagnosis = patient_dict[image_filename]
        if target == 0:
            os.system('cp {}{}{} {}{}'.format(data_path, sub_path, file_name, data_path, 'abnormal/'))
        if target == 1:
            os.system('cp {}{}{} {}{}'.format(data_path, sub_path, file_name, data_path, 'normal/'))

In [5]:
class Xray(data.Dataset):
    def __init__(self, main_path, img_size=64, transform=None):
        super(Xray, self).__init__()
        self.transform = transform
        self.file_path = []
        self.labels = []
        self.slices = []
        self.transform = transform if transform is not None else lambda x: x

        for label in os.listdir(main_path):
            if label not in ['0', '1']:
                continue
            for file_name in tqdm(os.listdir(main_path + '/' + label)):
                data = sitk.ReadImage(main_path + '/' + label + '/' + file_name)
                data = sitk.GetArrayFromImage(data).squeeze()
                img = Image.fromarray(data).convert('L').resize((img_size, img_size), resample=Image.BILINEAR)
                self.slices.append(img)
                self.labels.append(int(label))

    def __getitem__(self, index):
        img = self.slices[index]
        label = self.labels[index]
        img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.slices)

In [6]:
def get_xray_dataloader(bs, workers, dtype='train', img_size=64, dataset='train_humerus'):
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    if dataset == 'train_humerus':
        path = DATA_PATH + 'train_humerus_xray/'
    elif dataset == 'test_humerus':
        path = DATA_PATH + 'test_humerus_xray/'

    path += dtype

    dset = Xray(main_path=path, transform=transform, img_size=img_size)
    train_flag = True if dtype == 'train' else False
    dataloader = data.DataLoader(dset, bs, shuffle=train_flag,
                                 drop_last=train_flag, num_workers=workers, pin_memory=True)

    return dataloader

In [7]:
if __name__ == '__main__':
    read_data()

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]


KeyError: 'image1'